On this homework, you will see the need for *pivoting* (or row swaps) in row reduction algorithms. Two significant (and related) issues arise when row reducing. The first two questions explore these issues. For both of these questions, work without any row swaps.

> ## Make a copy of this notebook (File menu -> Make a Copy...)

### Homework Question 1

Consider the matrix $$\begin{bmatrix} 1 & 2 & 3 & -2\\ 2 & 4 & 1 &  0\\ 3 & 3 & 2 & 5 \\ -1 & 6 & 2 & 1\end{bmatrix}.$$ Row-reduce this matrix by hand, then using your `rowred()` code from lab. What happens? Explain why the issue occurs.

This error occurs because without row swapping, there is a row with a pivot in it in the wrong position. Because the code assumes that the row directly above the row whose first value we are trying to reduce to zero has a nonzero value in that same column (here, the second value in the second row is a zero, not a nonzero value) the program ends up trying to divide the row by zero in order to reduce the next row to zero. Thus, this causes an error by dividing by zero.

In [2]:
import numpy as np

def swaprows(A,i,j):
    A[[i, j],:] = A[[j, i],:] 
def rowmult(A,i,c):
    A[i] = A[i]*c
def rowaddmult(A,i,j,c):
    A[j] = A[i]*c + A[j]
def rowred(A):
    rows,cols = A.shape
    copy = A.copy()
    pivotcol = 0
    pivotrow = 0
    i = 1
    while((pivotcol<cols) & (pivotrow<rows)):
        while(i<rows):
            rowaddmult(copy,pivotrow,i,((-1*copy[i,pivotcol])/(copy[pivotrow,pivotcol])))
            i+=1
        pivotcol+=1
        pivotrow+=1
        i = pivotrow+1
    return copy;

A = np.array([[1.,2,3,-2],[2,4,1,0],[3,3,2,5],[-1,6,2,1]])
print(rowred(A))

[[  1.   2.   3.  -2.]
 [  0.   0.  -5.   4.]
 [ nan  nan -inf  inf]
 [ nan  nan  nan  nan]]


C:\Users\miche\anaconda3\envs\dsenv\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\miche\anaconda3\envs\dsenv\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in multiply
  
C:\Users\miche\anaconda3\envs\dsenv\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars


### Homework Question 2

Consider the following system of three simultaneous equations in three variables:

$$\begin{align*} 0.0001x_1 &+ 10,000x_3 &&= 10,000.0001 \\ 10,000x_1 &+ 0.0001x_2 &&= 10,000.0001\\10,000x_2 &+ x_3 &&= 10,001\end{align*}$$

1. Write this system as a matrix equation $Ax=v$. 
1. Row-reduce the system and solve it by hand. You should get a nice mand simple answer. It may help to express everything as powers of 10, and to note that $$v=\begin{bmatrix}10^4+10^{-4}\\ 10^{4}+10^{-4}\\ 10^4+1\end{bmatrix}.$$ You might even be able to just spot the answer by looking at that, but row-reduce anyway. We'll need the hand-done row-reduction later in this homework.
1. Use your work from Question 11 in the lab to solve the system using NumPy. Do you get the right answer?

In [3]:
def backsub(U,v):
    rows,cols = U.shape
    x = np.zeros(cols)
    for i in range (rows-1,-1,-1):
        x[i] += (U[i, i+1:rows]@x[i+1:rows]) # dot product version 
        x[i] = v[i] - x[i]
        x[i] = x[i]/U[i,i]
    return x;

a = np.array([[0.0001,0,10000],[10000,.0001,0],[0,10000,1]])
v = np.array([10000.0001,10000.0001,10001])

def gauss(a,v):
    rows,cols = a.shape #dimensions of the original matrix yet to be augmented
    A = np.zeros(rows*(cols+1)).reshape(rows,cols+1)
    A[:rows,:cols] = a[:,:]
    A[:,cols] = v
    #print(A)
    sol = rowred(A)
    #print(sol)
    newsol = backsub(sol[:cols,:cols],sol[:,cols]) #here we have to separate the REF matrix into its augmented parts.
    return (newsol)
    
print(gauss(a,v))

[0.99999999 2.44140625 1.        ]


#### What's Going On?

The issue in the first question above is relatively simple: a zero appears in a pivot position. We cannot divide by zero, and so we're stuck.

The second question is more subtle. What happens there is an example of *floating point error*. While a full discussion of floating point error is beyond the scope of these labs, but the following question gives some insight:

### Homework Question 3

Look back at your hand-computed row-reduction from Homework Question 2. For each entry in the row-reduced augemented matrix, write down how many *significant figures* you'd need to write out each number in full. For example, the number 101 requires three significant figures, whereas 100 requires only one, since it can be written as $1e+02$ (i.e. $1\times 10^2$). Likewise, 100.001 ($1.00001e+02$) requires six, as does 0.000100001 ($1.00001e-4$).

Very briefly, floating point numbers can only hold a certain number of significant figures. Numbers requiring more than the limit are rounded. Run the following code, and use the output to find the maximum number of significant figures floats in Python can represent accurately. Explain your answer.
```python
for i in range(20):
    print(i,float(10**i+1)-10**i)
```

Lastly, let's look at where the incorrectly represented numbers came from. Go back to your row reduction, and find the exact places where Python could no longer represent numbers accurately. Explain the following sentence:
> *When we add two numbers of very different magnitudes, we may create numbers that cannot be accurately represented as floats.*

As you saw in Homework Question 2, it is possible to construct relatively simple examples where the limit is exceeded, resulting in very incorrect results that do not round correctly.

Note that we get a floating point issue within *A* itself, even without its augmented column. This will be important in the next lab.

Here, we can see that the maximum number of significant digits that floats can hold in Python is 15, because after that, we can see here that it loses its most insignificant digit and does not recall that it initially added 1 to the very large float (when that large float encompassed more than 15 digits).

Because floats in Python can only hold a certain number of digits and remain accurate, when two digits that are very disparate (like more than 15 significant digits apart from one another) are added together, the least significant one is rendered irrelevant and is assumed to be zero. We can see how this worked in the matrix above, because a very large number was added to a very small number, and this caused errors when we were using our code to reduce the matrix for the above reasons.

In [4]:
for i in range(20):
    print(i,float(10**i+1)-10**i)

0 1.0
1 1.0
2 1.0
3 1.0
4 1.0
5 1.0
6 1.0
7 1.0
8 1.0
9 1.0
10 1.0
11 1.0
12 1.0
13 1.0
14 1.0
15 1.0
16 0.0
17 0.0
18 0.0
19 0.0


To solve (or at least reduce) the problem we saw above, we use a strategy called *Maximal Partial Pivoting*. The idea is this: In a given row, look at all numbers in the column *below* the pivot. If there is a number whose magnitude (absolute value) is larger than the pivot, swap that row with the current one (if two rows have the same magnitude in that column, just pick the first to swap with). Then proceed with regular row reduction.

### Homework Question 4

By hand, carry out row reduction with MPP for the matrix $$\begin{bmatrix} 1 & 2 & 2 \\ 2 & 1 & 2 \\ 2 & -1 & 2\end{bmatrix}$$

$$\begin{bmatrix} 2 & 1 & 2 \\ 0 & -2 & 0 \\ 0 & 0 & 1\end {bmatrix}$$

### Homework Question 5

To find the index of the largest entry of a vector, use the command `np.argmax(v)`. Use this to modify your `rowred(A)` routine to create a new routine `rowredpivot(A)` that implements MPP. Test your code on the matrix from the last question, as well as the matrix from Homework Question 1.

**Note: When testing for a swap, be sure to only test entries below the current pivot. You will need to be a little careful with the output from `np.argmax()`.**

In [5]:
def swaprows(A,i,j):
    A[[i, j],:] = A[[j, i],:] 
def rowmult(A,i,c):
    A[i] = A[i]*c
def rowaddmult(A,i,j,c):
    A[j] = A[i]*c + A[j]
def rowredpivot(A):
    rows,cols = A.shape
    copy = A.copy()
    pivotcol = 0
    pivotrow = 0
    i = 1
    while((pivotcol<cols) & (pivotrow<rows)):
        while(i<rows):
            maxe = np.argmax(abs(copy[:,pivotcol]))
            if (maxe > pivotrow):
                copyrow = (copy[pivotrow]).copy();
                copy[pivotrow] = (copy[maxe]).copy();
                copy[maxe] = copyrow;
            rowaddmult(copy,pivotrow,i,((-1*copy[i,pivotcol])/(copy[pivotrow,pivotcol])))
            i+=1
        pivotcol+=1
        pivotrow+=1
        i = pivotrow+1
    return copy;
#A = np.array([[1,2.,2],[2,1,2],[2,-1,2]])
A = np.array([[1.,2,3,-2],[2,4,1,0],[3,3,2,5],[-1,6,2,1]])
print(rowredpivot(A))



[[ 3.          3.          2.          5.        ]
 [ 0.          7.          2.66666667  2.66666667]
 [ 0.          0.          1.95238095 -4.04761905]
 [ 0.          0.          0.         -6.36585366]]


### Homework Question 6

Repeat Homework Question 2 above with your new routine. You should get the right answer this time. Lastly, carry out row-reduction with MPP by hand for this system. You should still find that there are places where we get rounding errors. Can you explain why the answer you get from your routine is nonetheless correct? We will explore this more in depth on the next homework.

In [7]:
a = np.array([[0.0001,0,10000],[10000,.0001,0],[0,10000,1]])
v = np.array([10000.0001,10000.0001,10001])

def gauss(a,v):
    rows,cols = a.shape #dimensions of the original matrix yet to be augmented
    A = np.zeros(rows*(cols+1)).reshape(rows,cols+1)
    A[:rows,:cols] = a[:,:]
    A[:,cols] = v
    #print(A)
    sol = rowredpivot(A)
    print(sol)
    newsol = backsub(sol[:cols,:cols],sol[:,cols]) #here we have to separate the REF matrix into its augmented parts.
    return (newsol)

print(gauss(a,v))

[[1.00000000e+04 1.00000000e-04 0.00000000e+00 1.00000001e+04]
 [0.00000000e+00 1.00000000e+04 1.00000000e+00 1.00010000e+04]
 [0.00000000e+00 0.00000000e+00 1.00000000e+04 1.00000000e+04]]
[1. 1. 1.]


Even though there are some rounding errors, the answer from this routine is still correct. Even though the ref matrices do not end up being the same due to differences in the way that it was reduced order-wise, the final answer is stil the same because there can be several different REF matrices for any given matrix, and solving with back substitution resolves this issue.